In [1]:
import sys
sys.path.append('C:/Users/visha/Documents/GitHub/Club_Mahindra_Data_Hack/Club_Mahindra_Data_Hack')

# Import standard libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
color = sns.color_palette()

# Import local libraries
from src.models import train_models

pd.options.display.max_columns = 100
import warnings
warnings.filterwarnings("ignore")

### Load Data

In [2]:
train_preprocessed = pd.read_csv(r'../data/processed/preprocessed_data/train_preprocessed.csv')
val_preprocessed = pd.read_csv(r'../data/processed/preprocessed_data/val_preprocessed.csv')
test_preprocessed = pd.read_csv(r'../data/processed/preprocessed_data/test_preprocessed.csv')

In [3]:
train_target = pd.read_csv(r'../data/processed/target_values/train_target_values.csv', header=None)
val_target = pd.read_csv(r'../data/processed/target_values/val_target_values.csv', header=None)
test_target = pd.read_csv(r'../data/processed/target_values/test_target_values.csv', header=None)

### Build and Train Models On Training Set 

In [4]:
adb_reg, train_adb_score = train_models.train_adb(train_preprocessed,train_target)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 37.5min finished


1.1515561433586956


In [5]:
etr_reg, train_etr_score = train_models.train_etr(train_preprocessed,train_target)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 13.2min finished


1.024060889811498


In [6]:
xgb_reg, train_xgb_score = train_models.train_xgb(train_preprocessed,train_target)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  7.2min finished


0.9999955347929


In [13]:
# save the models to disk
adb_filename = r'../models/adaboost_model.sav'
etr_filename = r'../models/extratrees_model.sav'
xgb_filename = r'../models/xgboost_model.sav'

pickle.dump(adb_reg, open(adb_filename, 'wb'))
pickle.dump(etr_reg, open(etr_filename, 'wb'))
pickle.dump(xgb_reg, open(xgb_filename, 'wb'))

### Stack and Blend Models
Stack the above 3 models and blend them using a Random Forest Regressor. Then train the blender on the validation set

In [7]:
blender_model, blender_score = train_models.stack_models(val_preprocessed, val_target, xgb_reg, adb_reg, etr_reg)

1.091667081256894


### Evaluate The Best Model On Test Set

In [8]:
# Create a table of all the models with their mean cross-validated scores
scoreboard = pd.DataFrame({'Model_Name': ['XGBoost Regressor', 'AdaBoost Regressor', 'Extra Trees Regressor', 'RF Blender'],
                           'Model': [xgb_reg, adb_reg, etr_reg, blender_model],
                           'Score': [train_xgb_score, train_adb_score, train_etr_score, blender_score]})

winner_model_name = scoreboard.loc[scoreboard[['Score']].idxmin()]['Model_Name']
winner_model = scoreboard.loc[scoreboard[['Score']].idxmin()]['Model']

print (scoreboard)
print('{} model performs the best on Validation Set'.format(winner_model_name[0]))

              Model_Name                                              Model  \
0      XGBoost Regressor  XGBRegressor(base_score=0.5, booster='gbtree',...   
1     AdaBoost Regressor  (DecisionTreeRegressor(criterion='mse', max_de...   
2  Extra Trees Regressor  (ExtraTreeRegressor(criterion='mse', max_depth...   
3             RF Blender  RandomForestRegressor(bootstrap=True, criterio...   

      Score  
0  0.999996  
1  1.151556  
2  1.024061  
3  1.091667  
XGBoost Regressor model performs the best on Validation Set


In [9]:
# Check the Root of Winner Model on Test set
final_score = train_models.evaluate_on_test(winner_model[0], test_preprocessed, test_target)

In [14]:
print('The final Root Mean Squared Error (RMSE score) of {} on unseen data is {}'.format(winner_model_name[0], np.round(final_score, 2)))

The final Root Mean Squared Error (RMSE score) of XGBoost Regressor on unseen data is 1.34


### Conclusion
Here, I compared three ensemble algorithms, viz. XGBoost, AdaBoost and Extremely Randomized Trees. I also stacked these models and blended them using a Random Forest Regressor. Though XGBoost and Extra Trees Regressors have comparable performances, XGBoost performed better than all others.

Also, these models have been trained on a subset of the original data (size of the dataset is 60k rows, considering the processing/ memory capabilities of my machine). If we train the models on a larger size of original data or the entire original data then the performance of these models would certainly improve.

I will update the models and their scores once I get train it on the entire dataset.

## Thanks for reading!